In [235]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import datetime as dt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics as mt
pd.set_option('display.max_columns', None)

In [236]:
############################## for Experiment-1 use 1083 accounts for genuine accounts dataset #################################
from numpy import loadtxt
os.chdir("/Users/Lenovo/")
ids = loadtxt("genuine_account_ids.txt", delimiter="\n", unpack=False)
ids
################################################################################################################################

array([6.78033000e+05, 7.22623000e+05, 7.55116000e+05, ...,
       2.35423776e+09, 2.35584778e+09, 2.35642575e+09])

In [237]:
rus_users = pd.read_csv('/xxx/dataset/rus_users.csv',na_filter=False,) #fill directory
rus_users.fillna('')
rus_users[['id','followers_count','statuses_count','favourites_count','friends_count']] = rus_users[['id','followers_count','statuses_count','favourites_count','friends_count']].apply(pd.to_numeric)
rus_users['id'] = rus_users['id'].fillna(0).astype(int)
rus_users['followers_count'] = rus_users['followers_count'].fillna(0).astype(int)
rus_users['statuses_count'] = rus_users['statuses_count'].fillna(0).astype(int)
rus_users['favourites_count'] = rus_users['favourites_count'].fillna(0).astype(int)
rus_users['friends_count'] = rus_users['friends_count'].fillna(0).astype(int)
rus_users.fillna('')
rus_users = rus_users.replace(np.nan, '', regex=True)
rus_users['knownbot'] = 1

In [238]:
os.chdir("/xxx/dataset/")

real = pd.read_csv('genuine_accounts_users.csv')
real = real.fillna('')
real['knownbot'] = 0
#real =real[real['id'].isin(ids)] #use for Experiment-1 -> 1083 account, commented out for experiment

In [239]:
fake_followers = pd.read_csv('fake_followers_users.csv')
fake_followers = fake_followers.fillna('')

trad_spam_1 = pd.read_csv('traditional_spambots_1_users.csv')
trad_spam_2 = pd.read_csv('traditional_spambots_2_users.csv')
trad_spam_3 = pd.read_csv('traditional_spambots_3_users.csv')
trad_spam_4 = pd.read_csv('traditional_spambots_4_users.csv')
trad_spam_1 = trad_spam_1.fillna('')
trad_spam_2 = trad_spam_2.fillna('')
trad_spam_3 = trad_spam_3.fillna('')
trad_spam_4 = trad_spam_4.fillna('')

social_spam_1 = pd.read_csv('social_spambots_1_users.csv')
social_spam_2 = pd.read_csv('social_spambots_2_users.csv')
social_spam_3 = pd.read_csv('social_spambots_3_users.csv')
social_spam_1 = social_spam_1.fillna('')
social_spam_2 = social_spam_2.fillna('')
social_spam_3 = social_spam_3.fillna('')

fake_followers['knownbot'] = 1
trad_spam_1['knownbot'] = 1
trad_spam_2['knownbot'] = 1
trad_spam_3['knownbot'] = 1
trad_spam_4['knownbot'] = 1
social_spam_1['knownbot'] = 1
social_spam_2['knownbot'] = 1
social_spam_3['knownbot'] = 1


#concat according to experiment and scenario
all_trad_spambots = pd.concat([trad_spam_1, trad_spam_2, trad_spam_3, trad_spam_4],sort=False)
all_social_spambots = pd.concat([social_spam_1, social_spam_3],sort=False)
all_bots = pd.concat([all_social_spambots,all_trad_spambots,fake_followers, rus_users],sort=False)
train_dataset = pd.concat([real, all_bots],sort=False)

#FOR EXPERIMENT 3 COMMENT OUT ABOVE DATASETS AND USE THIS
#os.chdir('/Users/Lenovo/dataset')
#train_dataset1 = pd.read_csv("politicalBots.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset2 = pd.read_csv("botometerFeedback.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset = pd.read_csv("gilani.csv", na_filter=False, lineterminator='\n', sep=',')
#train_dataset = pd.concat([train_dataset1,train_dataset2])

train_dataset = train_dataset.fillna('').sort_values(by='id')

In [240]:
#implementing the features

train_score = pd.DataFrame()
train_score['id'] = train_dataset['id']
train_score['lang-en'] = train_dataset['lang'].map({'en': 0, None: 1})
train_score['profile_pic'] = train_dataset['default_profile_image'].map({True:0, False:1})
train_score['has_screen_name'] = train_dataset['name'].map(lambda x: 1 if x == '' else 0)
train_score['30followers'] = train_dataset['followers_count'].map(lambda x:1 if x<30 else 0)
train_score['geoloc'] = train_dataset['geo_enabled'].map({True:0, False:1})
train_score['banner_link'] = train_dataset['profile_banner_url'].map(lambda x:0 if x=='' else 1)
train_score['50tweets'] = train_dataset['statuses_count'].map(lambda x:0 if x>50 else 1)
train_score['twice_num_followers'] = train_dataset['id'].map(lambda x: 0 if 2*train_dataset.loc[train_dataset['id'] == x]['followers_count'].all() >= train_dataset.loc[train_dataset['id'] == x]['friends_count'].all() else 1)
train_score['1000friends'] = train_dataset['friends_count'].map(lambda x:1 if x>1000 else 0)
train_score['NeverTweeted'] = train_dataset['statuses_count'].map(lambda x:1 if x==0 else 0)
train_score['fifty_FriendsFollowersRatio'] = train_dataset['id'].map(lambda x: 0 if 50*train_dataset.loc[train_dataset['id'] == x]['followers_count'].all() > train_dataset.loc[train_dataset['id'] == x]['friends_count'].all() else 1)
train_score['hundred_FriendsFollowersRatio'] = train_dataset['id'].map(lambda x: 0 if 100*train_dataset.loc[train_dataset['id'] == x]['followers_count'].all() > train_dataset.loc[train_dataset['id'] == x]['friends_count'].all() else 1)
train_score['has_description'] =train_dataset['description'].map(lambda x: 1 if x == '' else 0)
train_score = train_score.fillna('0')
train_score

,id,lang-en,profile_pic,has_screen_name,30followers,geoloc,banner_link,50tweets,twice_num_followers,1000friends,NeverTweeted,fifty_FriendsFollowersRatio,hundred_FriendsFollowersRatio,has_description
2274,678033,0,0,0,0,0,1,0,0,1,0,0,0,0
257,722623,0,0,0,0,0,1,0,0,0,0,0,0,0
2253,755116,0,0,0,0,0,1,0,0,0,0,0,0,0
3111,755746,0,0,0,0,0,1,0,0,0,0,0,0,0
3370,785080,0,0,0,0,0,1,0,0,0,0,0,0,0
1947,806585,0,0,0,0,0,0,0,0,0,0,0,0,0
2819,806975,0,0,0,0,0,1,0,0,0,0,0,0,0
1123,887281,0,0,0,0,0,1,0,0,0,0,0,0,0
2485,1382561,0,0,0,0,0,1,0,0,1,0,0,0,0
1143,1813221,0,0,0,0,0,1,0,0,0,0,0,0,0


In [241]:
train_score = train_score.iloc[:,1:]
train_score_y = train_dataset.iloc[:,42:].values.ravel()#.astype(np.float)
train_score = train_score.astype(float)

# EXPERIMENT 1

In [242]:
#In experiment 1 we split the data 80% to 20%
X_train, X_test, y_train, y_test = train_test_split(train_score, train_score_y, test_size=0.2, random_state=42) # random state for same split for all time

In [243]:
scl_obj = StandardScaler()
scl_obj.fit(X_train)

X_train_scaled = scl_obj.transform(X_train)
X_test_scaled = scl_obj.transform(X_test)

In [244]:
svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto')
svm_clf.fit(X_train_scaled, y_train)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [245]:
y_hat = svm_clf.predict(X_test_scaled) 

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('Accuracy:', acc)
print("Precision Score", precision_score(y_test, y_hat))
print("Recall Score", recall_score(y_test, y_hat))
print("F1 Score", f1_score(y_test, y_hat))
print("AUC", roc_auc_score(y_test, y_hat))
print("MCC", matthews_corrcoef(y_test,y_hat))
print(conf)

Accuracy: 0.90748031496063
Precision Score 0.9250814332247557
Recall Score 0.922077922077922
F1 Score 0.9235772357723576
AUC 0.903538961038961
MCC 0.8063778815145979
[[177  23]
 [ 24 284]]


# EXPERIMENT 2-3

In [158]:
os.chdir('/xxx/dataset')

test_dataset1 = pd.read_csv("vendorPurchased.csv", na_filter=False, lineterminator='\n', sep=',')
test_dataset2 = pd.read_csv("verified.csv", na_filter=False, lineterminator='\n', sep=',')
test_dataset3 = pd.read_csv("politicalBots.csv", na_filter=False, lineterminator='\n', sep=',')
#test_dataset = pd.read_csv("gilani.csv", na_filter=False, lineterminator='\n', sep=',')
test_dataset = pd.concat([test_dataset1, test_dataset2, test_dataset3])

test_dataset = test_dataset.fillna('')

In [160]:
test_score = test_score.iloc[:,1:]

In [161]:
test_score_y = test_dataset.iloc[:,42:].values.ravel()#.astype(np.float)

In [159]:
test_score = pd.DataFrame()
test_score['id'] = test_dataset['id']
test_score['lang-en'] = test_dataset['lang'].map({'en': 0, None: 1})
test_score['profile_pic'] = test_dataset['default_profile_image'].map({True:0, False:1})
test_score['has_screen_name'] = test_dataset['name'].map(lambda x: 1 if x == '' else 0)
test_score['30followers'] = test_dataset['followers_count'].map(lambda x:1 if x<30 else 0)
test_score['geoloc'] = test_dataset['geo_enabled'].map({True:0, False:1})
test_score['banner_link'] = test_dataset['profile_banner_url'].map(lambda x:0 if x=='' else 1)
test_score['50tweets'] = test_dataset['statuses_count'].map(lambda x:0 if x>50 else 1)
test_score['twice_num_followers'] = test_dataset['id'].map(lambda x: 0 if 2*test_dataset.loc[test_dataset['id'] == x]['followers_count'].all() >= test_dataset.loc[test_dataset['id'] == x]['friends_count'].all() else 1)
test_score['1000friends'] = test_dataset['friends_count'].map(lambda x:1 if x>1000 else 0)
test_score['NeverTweeted'] = test_dataset['statuses_count'].map(lambda x:1 if x==0 else 0)
test_score['fifty_FriendsFollowersRatio'] = test_dataset['id'].map(lambda x: 0 if 50*test_dataset.loc[test_dataset['id'] == x]['followers_count'].all() > test_dataset.loc[test_dataset['id'] == x]['friends_count'].all() else 1)
test_score['hundred_FriendsFollowersRatio'] = test_dataset['id'].map(lambda x: 0 if 100*test_dataset.loc[test_dataset['id'] == x]['followers_count'].all() > test_dataset.loc[test_dataset['id'] == x]['friends_count'].all() else 1)
test_score['has_description'] =test_dataset['description'].map(lambda x: 1 if x == '' else 0)
test_score = test_score.fillna('0')
test_score

,id,lang-en,profile_pic,has_screen_name,30followers,geoloc,banner_link,50tweets,twice_num_followers,1000friends,NeverTweeted,fifty_FriendsFollowersRatio,hundred_FriendsFollowersRatio,has_description
0,1069351,0,1,0,0,1,1,0,0,1,0,0,0,0
1,14483001,0,1,0,0,0,1,0,0,1,0,0,0,0
2,14993272,0,1,0,0,1,1,0,0,1,0,0,0,0
3,15210670,0,1,0,0,0,1,0,0,1,0,0,0,0
4,16655296,0,1,0,0,0,1,0,0,1,0,0,0,0
5,17059993,0,1,0,0,0,1,0,0,1,0,0,0,0
6,17617356,0,1,0,0,1,1,0,0,1,0,0,0,0
7,18343393,0,1,0,0,0,1,0,0,1,0,0,0,0
8,18992077,0,1,0,0,1,1,0,0,1,0,0,0,0
9,19199038,0,1,0,0,0,1,0,0,1,0,0,0,0


In [162]:
scl_obj = StandardScaler()
scl_obj.fit(train_score.astype(float))

train_score = train_score.astype(float)
test_score= test_score.astype(float)

X_train_scaled = scl_obj.transform(train_score)
X_test_scaled = scl_obj.transform(test_score)

In [163]:
svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto') # get object
svm_clf.fit(X_train_scaled, train_score_y) # train object

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [164]:
y_hat = svm_clf.predict(X_test_scaled) 

acc = mt.accuracy_score(test_score_y,y_hat)
conf = mt.confusion_matrix(test_score_y,y_hat)
print('Accuracy:', acc)
print("Precision Score", precision_score(test_score_y, y_hat))
print("Recall Score", recall_score(test_score_y, y_hat))
print("F1 Score", f1_score(test_score_y, y_hat))
print("AUC", roc_auc_score(test_score_y, y_hat))
print(conf)

Accuracy: 0.7759005419190309
Precision Score 0.8768971332209107
Recall Score 0.45217391304347826
F1 Score 0.5966724039013196
AUC 0.7077175554145424
[[1914   73]
 [ 630  520]]
